### Run upon export from spreadsheet

In [1]:
import os

import numpy as np
import pandas as pd


f = '../mnt/tess/labels/triage-v14.csv'

t = pd.read_csv(f, header=0, low_memory=False).set_index('Astro ID')

disps = ['E', 'J', 'N', 'S', 'B']
users = ['av', 'md', 'ch', 'as', 'mk', 'et', 'dm', 'td']
for d in disps:
  t[f'disp_{d}'] = 0

def set_labels(row):
  a = ~row.isna()
  if a['Final'] and row['Final'] in disps:
    row[f'disp_{row["Final"]}'] = 1
  else:
    for user in users:
      if a[user] and row[user] in disps:
        row[f'disp_{row[user]}'] += 1
  return row
t = t.apply(set_labels, axis=1)

t_train = t[t['Split'] == 'train']
t_val = t[t['Split'] == 'val']
t_test = t[t['Split'] == 'test']

assert not any((t_train['disp_E'] + t_train['disp_J']+ t_train['disp_N'] + t_train['disp_S'] + t_train['disp_B']) == 0)
assert not any((t_val['disp_E'] + t_val['disp_J']+ t_val['disp_N'] + t_val['disp_S'] + t_val['disp_B']) == 0)
assert not any((t_test['disp_E'] + t_test['disp_J']+ t_test['disp_N'] + t_test['disp_S'] + t_test['disp_B']) == 0)

print('Splits')
print('  train:', len(t_train))
print('  val:', len(t_val))
print('  test:', len(t_test))

t_train.to_csv('../mnt/tess/astronet/tces-v14-train.csv')
t_val.to_csv('../mnt/tess/astronet/tces-v14-val.csv')
t_test.to_csv('../mnt/tess/astronet/tces-v14-test.csv')
t.to_csv('../mnt/tess/astronet/tces-v14-all.csv')

Splits
  train: 19919
  val: 2491
  test: 2516


In [2]:
t

,TIC ID,Final,Decision,av,md,ch,as,mk,et,dm,...,Split,Year,MaxT,MinT,File,disp_E,disp_J,disp_N,disp_S,disp_B
Astro ID,,,,,,,,,,,,,,,,,,,,,
1,290603338,NaN,NaN,J,N,J,NaN,NaN,NaN,NaN,...,train,1,1682.344324,1624.969341,tess2019247000000-0000000290603338-111-cr_llc....,0,2,1,0,0
2,32092337,J,NaN,J,J,J,NaN,NaN,NaN,NaN,...,train,1,1652.863937,1325.322602,tess2019247000000-0000000032092337-111-cr_llc....,0,1,0,0,0
3,278544052,J,NaN,J,J,J,NaN,NaN,NaN,NaN,...,train,1,1682.344514,1596.781910,tess2019247000000-0000000278544052-111-cr_llc....,0,1,0,0,0
4,380752037,J,NaN,J,J,J,NaN,NaN,NaN,NaN,...,train,1,1682.343003,1653.926229,tess2019247000000-0000000380752037-111-cr_llc....,0,1,0,0,0
5,259863095,J,NaN,J,J,J,NaN,NaN,NaN,NaN,...,train,1,1682.342880,1325.323285,tess2019247000000-0000000259863095-111-cr_llc....,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28939,99349315,J,J,NaN,NaN,J,J,J,J,J,...,train,3,2254.063155,1491.640311,mk_hlsp_qlp_tess_ffi-s0034-0000000099349315_te...,0,1,0,0,0
28940,99738392,J,NaN,NaN,NaN,J,J,J,J,J,...,train,3,2087.099563,1325.329124,mk_hlsp_qlp_tess_ffi-s0028-0000000099738392_te...,0,1,0,0,0
28941,99765092,J,NaN,NaN,NaN,J,J,J,J,J,...,train,3,2087.099576,1325.329116,mk_hlsp_qlp_tess_ffi-s0028-0000000099765092_te...,0,1,0,0,0


In [2]:
pd.set_option('display.max_columns', None)
t_train.sample(5)

,TIC ID,Final,Decision,av,md,ch,as,mk,et,dm,td,Notes,Epoc,Per,Dur,Depth,SMass,SRad,SRadEst,Tmag,RA,Dec,Split,Year,MaxT,MinT,File,disp_E,disp_J,disp_N,disp_S,disp_B
Astro ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
26639,144216851,NaN,NaN,NaN,B,J,B,B,B,B,B,NaN,1325.53388,0.13501,0.050042,272300,0.750,1.08383,1.065598,12.8870,328.398484,-45.268540,train,3,2087.098772,1325.328474,mk_hlsp_qlp_tess_ffi-s0028-0000000144216851_te...,0,1,0,0,6
27873,326770784,NaN,NaN,NaN,B,J,J,J,J,J,J,NaN,1656.96249,0.29167,0.029333,1240,1.640,3.05609,2.891894,10.0700,270.518033,-40.403425,train,3,2389.717531,1653.933114,mk_hlsp_qlp_tess_ffi-s0039-0000000326770784_te...,0,6,0,0,1
27948,345480042,J,NaN,NaN,J,J,J,J,J,J,J,NaN,1436.91700,13.16180,0.240625,105196,1.271,1.11348,1.316603,10.3205,62.531419,9.490645,train,3,2200.231076,1437.995711,mk_hlsp_qlp_tess_ffi-s0032-0000000345480042_te...,0,1,0,0,0
28504,468928376,J,NaN,NaN,NaN,J,J,J,J,J,J,NaN,1492.00787,0.44236,0.049333,1010,1.190,1.56647,1.495208,11.3009,118.337912,11.907646,train,3,2254.064512,1491.641504,mk_hlsp_qlp_tess_ffi-s0034-0000000468928376_te...,0,1,0,0,0
28832,83687552,NaN,NaN,NaN,NaN,J,J,J,J,B,J,NaN,1630.00314,1.60764,0.244417,2950,1.460,1.59200,1.618890,9.0025,249.630114,-41.635281,train,3,2389.717184,1624.974541,mk_hlsp_qlp_tess_ffi-s0039-0000000083687552_te...,0,5,0,0,1


In [3]:
t_val.sample(5)

,TIC ID,Final,Decision,av,md,ch,as,mk,et,dm,td,Notes,Epoc,Per,Dur,Depth,SMass,SRad,SRadEst,Tmag,RA,Dec,Split,Year,MaxT,MinT,File,disp_E,disp_J,disp_N,disp_S,disp_B
Astro ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
27890,332911680,NaN,NaN,NaN,B,J,J,J,J,J,J,NaN,1411.48715,0.67708,0.107667,32650,0.934,1.39200,1.357377,9.91741,35.149175,-0.686904,val,3,2169.946353,1410.933227,mk_hlsp_qlp_tess_ffi-s0031-0000000332911680_te...,0,6,0,0,1
28235,406277413,J,NaN,NaN,J,J,J,J,J,J,J,NaN,1340.53093,23.61695,0.266625,111600,1.040,1.19979,1.116176,13.31690,326.227306,-62.047269,val,3,2087.097612,1325.327661,mk_hlsp_qlp_tess_ffi-s0028-0000000406277413_te...,0,1,0,0,0
28103,382199441,J,NaN,NaN,J,J,J,J,J,J,J,NaN,1326.24480,1.75069,0.162458,80,1.430,2.46657,2.498157,9.44990,81.538269,-57.018279,val,3,2389.711037,1325.322993,mk_hlsp_qlp_tess_ffi-s0039-0000000382199441_te...,0,1,0,0,0
28833,842068824,J,J,NaN,NaN,E,J,J,E,J,J,NaN,2256.47542,0.88680,0.032792,3110,1.190,1.95319,1.717433,12.15200,141.634139,-2.846360,val,3,2279.981060,2254.995267,mk_hlsp_qlp_tess_ffi-s0035-0000000842068824_te...,0,1,0,0,0
27007,200323039,S,NaN,NaN,S,S,S,S,S,S,S,NaN,1396.22031,34.66226,0.137375,214321,0.990,1.34173,1.374656,12.54440,76.508963,-46.675470,val,3,2200.227837,1410.929312,mk_hlsp_qlp_tess_ffi-s0032-0000000200323039_te...,0,0,0,1,0


In [4]:
t_test.sample(5)

,TIC ID,Final,Decision,av,md,ch,as,mk,et,dm,td,Notes,Epoc,Per,Dur,Depth,SMass,SRad,SRadEst,Tmag,RA,Dec,Split,Year,MaxT,MinT,File,disp_E,disp_J,disp_N,disp_S,disp_B
Astro ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
27913,33831147,J,NaN,NaN,J,J,J,J,J,J,J,NaN,1332.67844,52.73903,0.084375,1080,1.05,1.45417,1.446173,11.1013,63.471926,-73.786185,test,3,2389.712875,1325.324670,mk_hlsp_qlp_tess_ffi-s0039-0000000033831147_te...,0,1,0,0,0
26953,188989177,E,NaN,NaN,E,E,E,E,E,E,E,NaN,1438.29916,1.17409,0.102292,3200,2.02,3.66787,3.421632,9.1414,78.778406,-15.100456,test,3,2200.230382,1437.994331,mk_hlsp_qlp_tess_ffi-s0032-0000000188989177_te...,1,0,0,0,0
26645,144443966,J,NaN,NaN,J,J,J,J,J,J,J,NaN,1340.21061,49.83724,0.734583,16650,1.19,1.13175,1.124848,12.1814,335.956694,-45.822621,test,3,2087.098763,1325.328223,mk_hlsp_qlp_tess_ffi-s0028-0000000144443966_te...,0,1,0,0,0
26736,15373109,J,NaN,NaN,J,J,J,J,J,J,J,NaN,1569.76806,13.66458,0.168833,5833920,1.21,1.83081,1.835633,13.0468,185.233809,-52.541057,test,3,2332.577208,1569.452105,mk_hlsp_qlp_tess_ffi-s0037-0000000015373109_te...,0,1,0,0,0
28771,73458785,J,NaN,NaN,NaN,J,J,J,J,J,J,NaN,1571.90152,0.47014,0.065750,760,1.06,1.18066,1.173883,10.2145,191.010216,-46.454850,test,3,2332.577685,1569.452532,mk_hlsp_qlp_tess_ffi-s0037-0000000073458785_te...,0,1,0,0,0
